In [1]:
import pandas
import numpy as np
import import_ipynb
import FeatureStore

Period = 40

importing Jupyter notebook from FeatureStore.ipynb


In [2]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe, scaler.mean_, scaler.scale_

In [ ]:
from sklearn.neural_network import MLPClassifier

sigma = 0
mu = 0
clf = []
def backtestStrategy(Porog = 0.5):
    global sigma
    global mu
    global clf
    #trying Alpha = 0.100000, period = 40, function = relu
    period = 40
    alpha = 0.1
    activation = 'relu'
    
    data = pandas.read_csv('SPY2019_ForTrain.csv', delimiter=';')
    date = data.iloc[:, 0]
    y_train = data.iloc[:,1]
    X_train = data.iloc[:, 2:46]
    
    #print(y_train[:10])
    y_train = pandas.Series(np.where(y_train.values >= 0, 1, 0),
          y_train.index)
    #print(y_train[:10])
    #print(y_train)
    #X_train = FeatureStore.getTrainDataset(period)
    #y_train = FeatureStore.getTrainTarget(period)
    
    X_train, sigma, mu = scaleDataFrame(X_train, debugPrint = False)
    #print(X_train)
    
    clf = MLPClassifier(hidden_layer_sizes=(100, 100), activation=activation, solver='sgd', 
                    alpha = alpha, max_iter = 100000, random_state=241)
    clf.fit(X_train, y_train)
    
    pass
    data = pandas.read_csv('SPY2019_ForTest.csv', delimiter=';')
    date = data.iloc[:, 0]
    Y = data.iloc[:,1]
    X = data.iloc[:, 2:46]
    
    probs = clf.predict(X)
    pass

backtestStrategy()
print(sigma)
print(mu)

#data = pandas.read_csv('SPY2019_ForTest.csv', delimiter=';')
#date = data.iloc[:, 0]
#Y = data.iloc[:,1]
#X = data.iloc[:, 2:46]

#X
#print(X)
#data

In [ ]:
print(sigma)
print(mu)

In [ ]:
import matplotlib.pyplot as plt
import math
data = pandas.read_csv('SPY2019_ForTest.csv', delimiter=';')
data['Date'] = pandas.to_datetime(data['Date'])
date = data.iloc[:, 0]
Y = data.iloc[:,1]
X = data.iloc[:, 2:46]

#print(X)
X = ((X-mu) / sigma)

probs = clf.predict_proba(X)
#plt.plot(probs[:,0])

#print(X.shape) #[622 rows x 44 columns]
#print(Y.shape)
#for index in range(0, date.size):
#    print("%s;%s;%s" %(date[index], probs[index,0], probs[index,1]))

In [ ]:
Prob0 = probs[:, 0]
Prob1 = probs[:, 1]

Prob0[Prob0.size - Period:Prob0.size] = 0
Prob1[Prob0.size - Period:Prob0.size] = 0
print(np.where(Prob0 >= 0.7, -1, 0)[0:10])
print(np.where(Prob1 >= 0.7, 1, 0)[0:10])

Position = np.where(Prob0 >= 0.7, -1, 0) + np.where(Prob1 >= 0.7, 1, 0)
print(Position[:10])
CumulativePosition = np.zeros(Position.size)
for index in range(0, Position.size):
    CumulativePosition[index] = np.sum(Position[max(index - Period,0) : index + 1])
    if (index < -1):
        print("index %d pos %d" % (index, CumulativePosition[index]))
df = (pandas.DataFrame({"Date": date, "CumulativePosition": CumulativePosition}))
#df.to_csv("test.csv", index=False, sep=";")

data2 = pandas.read_csv("SPY2019.csv")
data2 = data2[data2['Date'] >= '2017-03-30']
data2 = data2[data2['Date'] <= '2019-09-18']
data2.reset_index(drop = True, inplace = True)
data2['Date'] = pandas.to_datetime(data2['Date'])
#print(data2)

df["Profit"] = 0.0
for index in range(1, df.index.size):
    #print(index)
    pnl = df["Profit"]
    close = data2["Adj Close"]
    #print(close)
    pnl[index] = pnl[index - 1] + 100 * CumulativePosition[index] * (close[index] - close[index - 1])
#print(PnL)
df.to_csv("test.csv", index=False, sep=";")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(df['Date'], df['Profit'])

In [ ]:
date2 = df['Date'][621]
date1 = df['Date'][0]
print(type(date2))